In [1]:
!pip install basemap
!pip install basemap-data-hires

!pip install ipympl
!pip install nodejs-bin
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install jupyter-matplotlib
!pip install graphviz
!pip install shapely


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
An error occurred.
ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
See the log file for details:  /tmp/jupyterlab-debug-8mpv8cbx.log
An error occurred.
ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.or

In [3]:
import pandas as pd
import numpy as np
import importlib
from datetime import datetime
from datetime import timedelta
import time
import copy

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import math
import IPython
import os

In [4]:
base_lat = [32, 36]
base_lon = [125, 129]

# 탱커와 벌커의 평균 속도: 15~16노트(28~30km/h)
avg_speed = 25
pivot_time = 5
gridmap_pivot_distance_km = 5

In [8]:
TRAINING_DIR = 'training_data/'
TRAINING_CSV_LIST = ['Dynamic_20200204_13_0_0.csv',
                     'Dynamic_20200204_14_0_0.csv',
                     'Dynamic_20200204_15_0_0.csv',
                     'Dynamic_20200204_16_0_0.csv',
                     'Dynamic_20200204_17_0_0.csv']
data_set = pd.DataFrame()

In [9]:
for csv in TRAINING_CSV_LIST:
    file = TRAINING_DIR + csv
    print('Read file:%s'%file)
    if os.path.exists(file):
        tdf = pd.read_csv(file)
        data_set = pd.concat([data_set, tdf])
    else:
        print('%s is not exist.'%file)

Read file:training_data/Dynamic_20200204_13_0_0.csv
Read file:training_data/Dynamic_20200204_14_0_0.csv
Read file:training_data/Dynamic_20200204_15_0_0.csv
Read file:training_data/Dynamic_20200204_16_0_0.csv
Read file:training_data/Dynamic_20200204_17_0_0.csv


In [10]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [11]:
data_set

,Measurement_time,Predict_time_time,N_of_ships,Grid1_N_of_ships,Grid1_pos_weight,Grid1_avg_SOG,Grid2_N_of_ships,Grid2_pos_weight,Grid2_avg_SOG,Grid3_N_of_ships,...,Grid86_avg_SOG,Grid87_N_of_ships,Grid87_pos_weight,Grid87_avg_SOG,Grid88_N_of_ships,Grid88_pos_weight,Grid88_avg_SOG,Grid89_N_of_ships,Grid89_pos_weight,Grid89_avg_SOG
0,2020-02-04 15:00:00,2020-02-04 18:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
1,2020-02-04 17:00:00,2020-02-04 22:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,8.500000,2.0,1.0,0.000,11.0,0.0,0.00,0.0,0.0
2,2020-02-04 17:00:00,2020-02-04 22:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.0,0.0,0.000,2.0,0.0,0.00,0.0,0.0
3,2020-02-04 13:00:00,2020-02-04 14:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
4,2020-02-04 15:00:00,2020-02-04 18:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35875,2020-02-04 14:00:00,2020-02-04 16:00:00,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.000000,16.0,0.0,2.380,21.0,5.0,4.85,0.0,2.0
35876,2020-02-04 13:00:00,2020-02-04 14:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.000000,0.0,0.0,0.075,12.0,4.0,0.00,0.0,0.0
35877,2020-02-04 15:00:00,2020-02-04 18:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,6.0,0.0,7.800,4.0,1.0,0.00,0.0,0.0
35878,2020-02-04 14:00:00,2020-02-04 16:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0


### 우선 선박의 수로 예측을 진행하기로 함.

In [12]:
nmin = min(data_set['N_of_ships'].values)
nmax = max(data_set['N_of_ships'].values)

print('min of N : %d'%nmin)
print('max of N : %d'%nmax)

min of N : 0
max of N : 73


In [13]:
# space_itv = [0, 1, 2, 3, 5, 8, 13, 21, 34]
space_itv = [0, 5, 10, 15, 20, 30, 40, 50]
space = []

for i in range(0, len(space_itv)-1):
    tspace = [x for x in data_set['N_of_ships'].values if x >= space_itv[i] and x < space_itv[i+1]]
    space.append(tspace)
space.append([x for x in data_set['N_of_ships'].values if x >= space_itv[len(space_itv)-1]])

In [14]:
for s in space:
    print(len(s))

35240
387
88
61
54
23
14
13


In [15]:
### training에 사용되는 column들 ###
label = data_set['N_of_ships'].values
cols = list(data_set)[3:]
column_set = data_set[cols].astype(float)

In [16]:
label

array([0, 0, 0, ..., 0, 0, 0])

In [17]:
column_set

,Grid1_N_of_ships,Grid1_pos_weight,Grid1_avg_SOG,Grid2_N_of_ships,Grid2_pos_weight,Grid2_avg_SOG,Grid3_N_of_ships,Grid3_pos_weight,Grid3_avg_SOG,Grid4_N_of_ships,...,Grid86_avg_SOG,Grid87_N_of_ships,Grid87_pos_weight,Grid87_avg_SOG,Grid88_N_of_ships,Grid88_pos_weight,Grid88_avg_SOG,Grid89_N_of_ships,Grid89_pos_weight,Grid89_avg_SOG
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,8.500000,2.0,1.0,0.000,11.0,0.0,0.00,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.0,0.0,0.000,2.0,0.0,0.00,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.000000,16.0,0.0,2.380,21.0,5.0,4.85,0.0,2.0
35876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.000000,0.0,0.0,0.075,12.0,4.0,0.00,0.0,0.0
35877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,6.0,0.0,7.800,4.0,1.0,0.00,0.0,0.0
35878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0


In [18]:
### 변수들의 scale이 너무 다른 경우를 대비하여, scaling을 진행해준다 ###
# normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(column_set)
data_set_scaled = scaler.transform(column_set)
data_set_scaled

array([[-0.05902755, -0.12356861, -0.0481361 , ..., -0.32574262,
         0.        , -0.21875989],
       [-0.05902755, -0.12356861, -0.0481361 , ..., -0.32574262,
         0.        , -0.21875989],
       [-0.05902755, -0.12356861, -0.0481361 , ..., -0.32574262,
         0.        , -0.21875989],
       ...,
       [-0.05902755, -0.12356861, -0.0481361 , ..., -0.32574262,
         0.        , -0.21875989],
       [-0.05902755, -0.12356861, -0.0481361 , ..., -0.32574262,
         0.        , -0.21875989],
       [-0.05902755, -0.12356861, -0.0481361 , ...,  1.28806681,
         0.        ,  1.03409004]])

In [19]:
# split to train data and test data
n_train = int(0.9*data_set_scaled.shape[0]) # 트레이닝 데이터와 테스트 데이터의 수를 9:1 비율로 분할. 
train_data_scaled = data_set_scaled[0: n_train] 
train_dates = column_set[0: n_train]

test_data_scaled = data_set_scaled[n_train:]
test_dates = column_set[n_train:]
# print(test_dates.head(5))

In [20]:
print('Shape of train set:' + str(train_dates.shape))
print('Shape of test set:' + str(test_dates.shape))

Shape of train set:(32292, 267)
Shape of test set:(3588, 267)


In [21]:
# 하이퍼파라미터들
# data reformatting for LSTM
input_dim = 3  # input_dimension: training column 길이
# sequence length: 예측에 사용되는 data의 row길이. 이 경우에는 detected grid의 숫자로 train_dates.shape[1] / input_dim과 동일.
seq_len = int(train_dates.shape[1] / input_dim)

learning_rate = 0.01
epochs = 30
batch_size = 32
validation_split = 0.1
verbose = 1

In [27]:
trainX = []
trainY = []
testX = []
testY = []

In [28]:
for row in train_data_scaled:
    grid_info = [row[i*input_dim:(i+1)*input_dim] for i in range(0, int(len(row)/input_dim))]
    trainX.append(grid_info)
for row in test_data_scaled:
    grid_info = [row[i*input_dim:(i+1)*input_dim] for i in range(0, int(len(row)/input_dim))]
    testX.append(grid_info)
trainY = [[x] for x in label[0:n_train]]
testY = [[x] for x in label[n_train:]]

trainX, trainY = np.array(trainX), np.array(trainY)
testX, testY = np.array(testX), np.array(testY)

In [29]:
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(32292, 89, 3) (32292, 1)
(3588, 89, 3) (3588, 1)


In [25]:
# LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(trainX.shape[1], trainX.shape[2]), # (seq length, input dimension)
               return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(trainY.shape[1]))

model.summary()

2023-11-15 13:56:27.495357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-15 13:56:27.495977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-15 13:56:27.496335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-15 13:56:27.496595: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-15 13:56:27.498577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

IndexError: tuple index out of range

In [ ]:
# create an Adam optimizer with the specified learning rate
optimizer = Adam(learning_rate=learning_rate)
# compile your model using the custom optimizer
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
# Try to load weights
try:
    model.load_weights('./save_weights/lstm_weights.h5')
    print("Loaded model weights from disk")
except:
    print("No weights found, training model from scratch")
    # Fit the model
    history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size,
                    validation_split=validation_split, verbose=verbose)
    # Save model weights after training
    model.save_weights('./save_weights/lstm_weights.h5')

    plt.plot(history.history['loss'], label='Training loss')
    plt.plot(history.history['val_loss'], label='Validation loss')
    plt.legend()
    plt.show()

In [ ]:
# prediction
prediction = model.predict(testX)
print(prediction.shape, testY.shape)

In [ ]:
# generate array filled with means for prediction
mean_values_pred = np.repeat(scaler.mean_[np.newaxis, :], prediction.shape[0], axis=0)

In [ ]:
# substitute predictions into the first column
mean_values_pred[:, 0] = np.squeeze(prediction)


In [ ]:
# inverse transform
y_pred = scaler.inverse_transform(mean_values_pred)[:,0]
print(y_pred.shape)

In [ ]:
# generate array filled with means for testY
mean_values_testY = np.repeat(scaler.mean_[np.newaxis, :], testY.shape[0], axis=0)

In [ ]:
# substitute testY into the first column
mean_values_testY[:, 0] = np.squeeze(testY)

In [ ]:
# inverse transform
testY_original = scaler.inverse_transform(mean_values_testY)[:,0]
print(testY_original.shape)